# Processing the data in a way that the system can understand 

In [1]:
!pip install pandas

import pandas as pd
import os
import re

You should consider upgrading via the 'C:\Users\iliya\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


## Loading the requiert files
- Which are the clean user annotations, and 
- The .csv file with the links and names of the wikipedia articles

In [2]:
clean_annotations = pd.read_csv('./clean_data/clean_annotations_without_duplicates.csv', encoding='utf-8')
annotation_doc_list = pd.read_csv('./BG_Documents.csv', encoding='utf-8')

In [3]:
len(set(clean_annotations['doc_id']))

102

In [4]:
clean_annotations = clean_annotations[['doc_id','acronym', 'expansion','type', 'annotator']]
clean_annotations.drop_duplicates(keep='first', inplace=True, ignore_index=False)
clean_annotations.dropna(inplace=True)

In [5]:
out_expansion_df = clean_annotations[clean_annotations['type'] == 'out_expansion']
in_expansion_df = clean_annotations[clean_annotations['type'] == 'in_expansion']

In [6]:
print(in_expansion_df)
len(set(in_expansion_df['doc_id']))

                     doc_id acronym                           expansion  \
17                      UML     OMG             Object Management Group   
23                      UML     OMG             Object Management Group   
28      Административен_акт     АПК  Административнопроцесуалния кодекс   
34      Административен_акт     АПК  Административнопроцесуалния кодекс   
40     Акционерно_дружество      АД                Акционерно дружество   
...                     ...     ...                                 ...   
1434    Операционна_система     TOS               Tape Operating System   
1435    Операционна_система     TSS                 Time Sharing System   
1444    Операционна_система     ГПИ    Графичен Потребителски Интерфейс   
1458  Софтуер_с_отворен_код    FOSS       Free and open-source software   
1468  Софтуер_с_отворен_код     ОПЛ          Общ публичен лиценз на ГНУ   

              type                     annotator  
17    in_expansion    contact@iliyasgeorgiev.com

55

In [30]:
def extract_second_annotation(row):
    if 1 in row.keys():
        return row[1][0]
    else:
        return ""

in_expansion_df['In'] = in_expansion_df['acronym'] + ';' + in_expansion_df['expansion']+ str("\n")
in_expansion_df['In'] = in_expansion_df.groupby(['doc_id', 'annotator'])['In'].transform(lambda x: ''.join(x))
# in_expansion_df['In'] = in_expansion_df.groupby(['annotator'])['In'].transform(lambda x: ''.join(x))
in_expansion_df_sub = in_expansion_df[['doc_id', 'In', 'annotator']]
# print(in_expansion_df_sub)
# for row in in_expansion_df_sub.iterrows():
#     print(in_expansion_df_sub[row][0])

in_expansion_df_sub.reset_index(inplace=True)
in_expansion_df_sub.index.name = 'DocID'
# create new dataframe empty
in_expansion_df_new = pd.DataFrame(columns=['Title', 'WikiID', 'annotator'])
annotations = {}
# print (in_expansion_df_sub)
for row in in_expansion_df_sub.iterrows():
    # add annotations with key as doc_id and value as list of annotations
    if row[1]['doc_id'] in annotations:
        if row[1]['annotator'] in annotations[row[1]['doc_id']]:
            annotations[row[1]['doc_id']][row[1]['annotator']].append(row[1]['In'])
        else:
            annotations[row[1]['doc_id']][row[1]['annotator']] = [row[1]['In']]
    else:
        annotations[row[1]['doc_id']] = {}
        annotations[row[1]['doc_id']][row[1]['annotator']] = []
        annotations[row[1]['doc_id']][row[1]['annotator']] = [row[1]['In']]
# print (annotations)
ready_in = {}
u1 = []
u2 = []
other_counter = 0
for annotation in annotations:
    counter = 0
    other_counter = other_counter + 1
    ready_in[annotation] = {}
    for annotator in annotations[annotation]:
        ready_in[annotation][counter] = annotations[annotation][annotator]
        counter = counter + 1

# print (ready_in)
in_expansion_df_sub['Title'] = in_expansion_df_sub['doc_id'].replace('_', ' ', regex=True)
in_expansion_df_sub['WikiID'] = in_expansion_df_sub['Title'].apply(hash)
in_expansion_df_sub['WikiID'] = in_expansion_df_sub['WikiID'].astype(str).str[1:10]
# find doc id in the ready_in dictionary

# in_expansion_df_sub['doc_id'].apply(lambda x: print(ready_in[x][0]))
in_expansion_df_sub['U1 In'] = in_expansion_df_sub['doc_id'].apply(lambda x: ready_in[x][0][0] if ready_in[x][0] is not None else "")
## get the second annotator
in_expansion_df_sub['U2 In'] = in_expansion_df_sub['doc_id'].apply(lambda x: extract_second_annotation(ready_in[x]))

in_expansion_final = in_expansion_df_sub[['Title', 'WikiID', 'U1 In', 'U2 In']]

in_expansion_final = in_expansion_final.drop_duplicates(subset=['WikiID'], keep='first')
in_expansion_final



C:\Users\iliya\AppData\Local\Temp\ipykernel_11208\3299390588.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  in_expansion_df['In'] = in_expansion_df['acronym'] + ';' + in_expansion_df['expansion']+ str("\n")
C:\Users\iliya\AppData\Local\Temp\ipykernel_11208\3299390588.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  in_expansion_df['In'] = in_expansion_df.groupby(['doc_id', 'annotator'])['In'].transform(lambda x: ''.join(x))
C:\Users\iliya\AppData\Local\Temp\ipykernel_11208\3299390588.py:46: SettingW

,Title,WikiID,U1 In,U2 In
DocID,,,,
0,UML,469226920,OMG;Object Management Group\n,OMG;Object Management Group\n
2,Административен акт,691013543,АПК;Административнопроцесуалния кодекс\n,АПК;Административнопроцесуалния кодекс\n
4,Акционерно дружество,077230470,АД;Акционерно дружество\nЕАД;еднолично акционе...,АД;Акционерно дружество\nЕАД;еднолично акционе...
8,Американска психологична асоциация,877293298,АПА;Американската психологична асоциация\n,АПА;Американската психологична асоциация\n
10,Банкова консолидационна компания,581999707,БКК;Банковата консолидационна компания\n,БКК;Банковата консолидационна компания\n
12,Баркод,534298219,EAN;European Article Numbering\nUCC;Uniform Co...,EAN;European Article Numbering\nUCC;Uniform Co...
20,Берлин,815659232,ЕС;Европейския съюз\n,ЕС;Европейския съюз\n
22,Би Ти Ви,458901807,ДКД;Държавната комисия по далекосъобщения\nКРС...,ДКД;Държавната комисия по далекосъобщения\nКРС...
30,БНТ 2,943945613,БНТ;Българската национална телевизия\n,БНТ;Българската национална телевизия\n


In [37]:
out_expansion_df['Out'] = out_expansion_df['acronym'] + ';' + out_expansion_df['expansion']+ str("\n")
out_expansion_df['Out'] = out_expansion_df.groupby(['doc_id', 'annotator'])['Out'].transform(lambda x: ''.join(x))

out_expansion_df_sub = out_expansion_df[['doc_id', 'Out', 'annotator']]

out_expansion_df_sub.reset_index(inplace=True)
out_expansion_df_sub.index.name = 'DocID'

out_expansion_df_new = pd.DataFrame(columns=['Title', 'WikiID', 'annotator'])
annotations = {}
# print (out_expansion_df_sub)
for row in out_expansion_df_sub.iterrows():
    # add annotations with key as doc_id and value as list of annotations
    print (row[1])
    if row[1]['doc_id'] in annotations:
        if row[1]['annotator'] in annotations[row[1]['doc_id']]:
            annotations[row[1]['doc_id']][row[1]['annotator']].append(row[1]['Out'])
        else:
            annotations[row[1]['doc_id']][row[1]['annotator']] = [row[1]['Out']]
    else:
        annotations[row[1]['doc_id']] = {}
        annotations[row[1]['doc_id']][row[1]['annotator']] = []
        annotations[row[1]['doc_id']][row[1]['annotator']] = [row[1]['Out']]
# print (annotations)
ready_out = {}
u1 = []
u2 = []
other_counter = 0
for annotation in annotations:
    counter = 0
    other_counter = other_counter + 1
    ready_out[annotation] = {}
    for annotator in annotations[annotation]:
        ready_out[annotation][counter] = annotations[annotation][annotator]
        counter = counter + 1


out_expansion_df_sub['Title'] = out_expansion_df_sub['doc_id'].replace('_', ' ', regex=True)
# out_expansion_df_sub['WikiID'] = out_expansion_df_sub['doc_id'] 

out_expansion_df_sub['WikiID'] = out_expansion_df_sub['Title'].apply(hash)
out_expansion_df_sub['WikiID'] = out_expansion_df_sub['WikiID'].astype(str).str[1:10]

out_expansion_df_sub['U1 Out'] = out_expansion_df_sub['doc_id'].apply(lambda x: ready_out[x][0][0] if ready_out[x][0] is not None else "")
## get the second annotator
out_expansion_df_sub['U2 Out'] = out_expansion_df_sub['doc_id'].apply(lambda x: extract_second_annotation(ready_out[x]))

out_expansion_final = out_expansion_df_sub[['Title', 'WikiID', 'U1 Out', 'U2 Out']]

out_expansion_final = out_expansion_final.drop_duplicates(subset=['WikiID'], keep='first')
out_expansion_final

C:\Users\iliya\AppData\Local\Temp\ipykernel_11208\3021899135.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out_expansion_df['Out'] = out_expansion_df['acronym'] + ';' + out_expansion_df['expansion']+ str("\n")
C:\Users\iliya\AppData\Local\Temp\ipykernel_11208\3021899135.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out_expansion_df['Out'] = out_expansion_df.groupby(['doc_id', 'annotator'])['Out'].transform(lambda x: ''.join(x))


index                                                        0
doc_id                                       Conus_eugrammatus
Out          m;meter\n°C;Celsius\nКНР;Китайската народна ре...
annotator                               dsgeorgiev90@yahoo.com
Name: 0, dtype: object
index                                                        1
doc_id                                       Conus_eugrammatus
Out          m;meter\n°C;Celsius\nКНР;Китайската народна ре...
annotator                               dsgeorgiev90@yahoo.com
Name: 1, dtype: object
index                                                        2
doc_id                                       Conus_eugrammatus
Out          m;meter\n°C;Celsius\nКНР;Китайската народна ре...
annotator                               dsgeorgiev90@yahoo.com
Name: 2, dtype: object
index                                                        3
doc_id                                       Conus_eugrammatus
Out          m;meter\n°C;Celsius\nКНР;Китайската 

C:\Users\iliya\AppData\Local\Temp\ipykernel_11208\3021899135.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out_expansion_df_sub['Title'] = out_expansion_df_sub['doc_id'].replace('_', ' ', regex=True)


,Title,WikiID,U1 Out,U2 Out
DocID,,,,
0,Conus eugrammatus,555629930,m;meter\n°C;Celsius\nКНР;Китайската народна ре...,m;meter\n°C;Celsius\nКНР;Китайската народна ре...
8,JavaScript,913287079,ES6;ECMAScript 6\nHTML;HyperText Markup Langua...,ES6;ECMAScript 6\nHTML;HyperText Markup Langua...
17,UML,469226920,OMT;Object-modeling technique\nOOSE;Object-Ori...,IBM;International Business Machines Corporatio...
26,Административен акт,691013543,ДВ;Държавен вестник\nал.;алинея\nбр.;брой\nт.е...,ДВ;Държавен вестник\nал.;алинея\nбр.;брой\nт.е...
36,Американска психологична асоциация,877293298,САЩ;Съединените Американски Щати\nмлн.;милион\n,млн.;милион\n
...,...,...,...,...
1058,Христо Ботев,108321373,г.;година\nдр.;други\n,г.;година\nдр.;други\n
1062,Хълк,733804422,г.;година\n,г.;година\n
1064,Хърватия,780089199,km;kilometer\nkm2;Square kilometer\nkm²;Square...,km;kilometer\nkm2;Square kilometer\nkm²;Square...


# Combining In and out expansions in one Dataframe 

In [48]:
doc_titles_series = clean_annotations[['doc_id']].drop_duplicates(ignore_index=True)

# creating the hash keys
doc_titles_series['WikiID'] = doc_titles_series['doc_id'].apply(hash)
doc_titles_series['WikiID'] = doc_titles_series['WikiID'].astype(str).str[1:10]
doc_titles_series
doc_titles_series.rename({'doc_id':'Title'}, inplace=True, axis='columns')

# merge all Inexpansion and Out-expansion df's
expansion_df_final = doc_titles_series.merge(in_expansion_final, how='right', on='Title')
expansion_df_final= expansion_df_final.merge(out_expansion_final, how='right', on='Title')

# subset and rename columns
expansion_df_final = expansion_df_final[['Title', 'WikiID', 'U1 In', 'U1 Out', 'U2 In', 'U2 Out']]	

expansion_df_final.head()

,Title,WikiID,U1 In,U1 Out,U2 In,U2 Out
0,Conus eugrammatus,555629930,NaN,m;meter\n°C;Celsius\nКНР;Китайската народна ре...,NaN,m;meter\n°C;Celsius\nКНР;Китайската народна ре...
1,JavaScript,913287079,NaN,ES6;ECMAScript 6\nHTML;HyperText Markup Langua...,NaN,ES6;ECMAScript 6\nHTML;HyperText Markup Langua...
2,UML,469226920,OMG;Object Management Group\n,OMT;Object-modeling technique\nOOSE;Object-Ori...,OMG;Object Management Group\n,IBM;International Business Machines Corporatio...
3,Административен акт,691013543,АПК;Административнопроцесуалния кодекс\n,ДВ;Държавен вестник\nал.;алинея\nбр.;брой\nт.е...,АПК;Административнопроцесуалния кодекс\n,ДВ;Държавен вестник\nал.;алинея\nбр.;брой\nт.е...
4,Американска психологична асоциация,877293298,АПА;Американската психологична асоциация\n,САЩ;Съединените Американски Щати\nмлн.;милион\n,АПА;Американската психологична асоциация\n,млн.;милион\n


# Add the new 'right' wikiID's

In [50]:
expansion_df_final['WikiID'] = 0

# set rootdir to the location of the documents in the acx env.
rootdir = 'D:/University/Thesis/annotation_system/acx_annotation/documents/bg/Wikipedia_BG/'


# extract the Doc ID's
def getname(s):    
    nameregex=re.search('id\="(\d+)', s)     # Run search with regex
    
    if nameregex:                                  # If there is a match 
        string_sub = nameregex.group()
        wiki_id = string_sub[4:]
        return wiki_id


# func to get individule documents for the wikipedia data
def wiki_doc_finder(abs_path, list_of_doc):
    """
    
    """
    # creating the document paths
    doc_list = []
    
    for doc in list_of_doc:
        doc_list.append(str(doc.replace(' ', '_') + ".txt"))
    
    
    
    # fined the article ID's
    data = {'Title':[], 'WikiID':[]}
    
    for article in doc_list:
        f = open(str(rootdir+article), "r", encoding='utf-8')
        
        text_all = f.read()
        text_sub = text_all[:100]


        data['Title'].append(article[:-4])
        data['WikiID'].append(getname(text_sub))

    return data
                        
id_df = wiki_doc_finder(rootdir, expansion_df_final['Title'])
df = pd.DataFrame.from_dict(id_df)
df

expansion_df_final.drop(axis=1, labels=['WikiID'], inplace=True)
expansion_df_final = expansion_df_final.merge(df, how='left', on='Title')

expansion_df_final = expansion_df_final[['Title', 'WikiID', 'U1 In', 'U1 Out', 'U2 In', 'U2 Out']]
expansion_df_final

,Title,WikiID,U1 In,U1 Out,U2 In,U2 Out
0,Conus eugrammatus,NaN,NaN,m;meter\n°C;Celsius\nКНР;Китайската народна ре...,NaN,m;meter\n°C;Celsius\nКНР;Китайската народна ре...
1,JavaScript,63292,NaN,ES6;ECMAScript 6\nHTML;HyperText Markup Langua...,NaN,ES6;ECMAScript 6\nHTML;HyperText Markup Langua...
2,UML,125558,OMG;Object Management Group\n,OMT;Object-modeling technique\nOOSE;Object-Ori...,OMG;Object Management Group\n,IBM;International Business Machines Corporatio...
3,Административен акт,NaN,АПК;Административнопроцесуалния кодекс\n,ДВ;Държавен вестник\nал.;алинея\nбр.;брой\nт.е...,АПК;Административнопроцесуалния кодекс\n,ДВ;Държавен вестник\nал.;алинея\nбр.;брой\nт.е...
4,Американска психологична асоциация,NaN,АПА;Американската психологична асоциация\n,САЩ;Съединените Американски Щати\nмлн.;милион\n,АПА;Американската психологична асоциация\n,млн.;милион\n
...,...,...,...,...,...,...
95,Христо Ботев,NaN,БРЦК;Българския революционен централен комитет\n,г.;година\nдр.;други\n,БРЦК;Български революционен централен комитет\n,г.;година\nдр.;други\n
96,Хълк,376173,NaN,г.;година\n,NaN,г.;година\n
97,Хърватия,50904,ООН;Организацията на обединените нации\nСХС;съ...,km;kilometer\nkm2;Square kilometer\nkm²;Square...,ООН;Организацията на обединените нации\nСХС;съ...,km;kilometer\nkm2;Square kilometer\nkm²;Square...
98,Чип,64785,NaN,SiO2;Silicon dioxide\nнапр.;например\n,NaN,напр.;например\n


In [48]:
# document = 'Theorie'
# ID = 42038

# extra_row_1 = {'Title':[document], 'WikiID':[ID], 'In': ['NaN'], 'Out': ['NaN']}

# df_extra_1 = pd.DataFrame(extra_row_1)


# expansion_df_final = pd.concat([expansion_df_final, df_extra_1],  ignore_index=True)
# expansion_df_final

,Title,WikiID,In,Out
0,Soapserie,13343,"GTST;Goede tijden, slechte tijden\nONM;Onderwe...",ATWT;As the World Turns\nBNN;Bart's Neverendin...
1,Publiek_domein,35876,CC0;Creative Commons Publiek Domein Verklaring,CDSM;Copyright in the Digital Single Market\nC...
2,Bitumen,162682,PAK;polycyclische aromatische koolwaterstoffen,v.Chr.;voor Christus\n°C;graden celcius\nTg;gl...
3,Waterschapsverkiezingen,1642900,LTO;Land- en Tuinbouworganisatie\nT/m;Tot en m...,NaN
4,Edinburgh,35081,SNP;Scottish National Party,FC;Football Club\nSt;saint\nUNESCO;United Nati...
5,Verenigd_Koninkrijk,1813,EU;Europese Unie\nGB;Groot-Brittannië\nID-kaar...,BBC;British Broadcasting Corporation\nBSkyB;Br...
6,SBS6,31688,SBS6;Scandinavian Broadcasting System 6,NPO;Nederlandse Publieke Omroep\nRTL;Radio Tél...
7,Koekoeksgraad,4734912,NaN,DNA;Deoxyribonucleic acid
8,ASML,108617,ASM;Advanced Semiconductor Materials\nASMI;Adv...,AEX;Amsterdam Exchange\nAG;Aktiengesellschaft\...
9,Radiogolf,54754,RF;radiofrequentie,AF;audio frequency\nAM;Amplitudemodulatie\nELF...


# Saving all requiered files for the system

In [26]:
mapWikiIds = expansion_df_final[['WikiID']]
mapWikiIds.to_csv('D:/University/Thesis/acx/acrodisam_app/data/UsersWikipediaBG/mapWikiIds.txt',
                 index_label=False,
                 header=False)

In [51]:
expansion_df_final.to_csv('D:/University/Thesis/acx/acrodisam_app/data/UsersWikipediaBG/complete_user_annotations_new.csv', encoding='utf-8', index_label='DocID')

In [39]:
# acx_wiki = pd.read_csv('/Users/jesher/Desktop/Master data science UvA/Semester 2/Thesis/acx_104_pycharm/data/UsersWikipedia_en/complete_user_annotations.csv',
#                       index_col='DocID')

In [28]:
pd.set_option('display.max_rows', None)
len(expansion_df_final)

102